In [ ]:
!pip install transformers gradio wikipedia requests


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load public fake news model
tokenizer = AutoTokenizer.from_pretrained("Pulk17/Fake-News-Detection")
model = AutoModelForSequenceClassification.from_pretrained("Pulk17/Fake-News-Detection")

def classify_news(article):
    labels = ["REAL", "FAKE"]
    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred_idx = torch.argmax(probs).item()
        return {"label": labels[pred_idx], "score": float(probs[0][pred_idx])}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load public fake news model
tokenizer = AutoTokenizer.from_pretrained("Pulk17/Fake-News-Detection")
model = AutoModelForSequenceClassification.from_pretrained("Pulk17/Fake-News-Detection")

def classify_news(article):
    labels = ["REAL", "FAKE"]
    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred_idx = torch.argmax(probs).item()
        return {"label": labels[pred_idx], "score": float(probs[0][pred_idx])}


In [ ]:
# Groq Replace
GROQ_API_KEY = "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"


In [ ]:
def query_gemma(prompt):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gemma2-9b-it",  # <-- updated model
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    response = requests.post(url, headers=headers, json=payload)
    try:
        return response.json()["choices"][0]["message"]["content"]
    except Exception:
        return f"❌ Groq API error: {response.text}"


In [ ]:
def analyze_news(article):
    result = classify_news(article)
    label = result["label"]
    score = result["score"]

    prompt = f"""
You are a fact-checking expert.
Here is a news article:

\"{article}\"

The fake news model predicted: {label} (confidence: {score:.2f})

Please:
1. Explain why it is labeled as '{label.lower()}'.
2. If FAKE, cite Wikipedia-based counter-evidence.
3. If REAL, provide supporting info using Wikipedia.
"""
    explanation = query_gemma(prompt)
    return label, explanation


In [ ]:
import gradio as gr

gr.Interface(
    fn=analyze_news,
    inputs=gr.Textbox(lines=10, label="Paste News Article Here"),
    outputs=[
        gr.Label(label="Prediction"),
        gr.Textbox(label="Gemma's Explanation + Wikipedia Evidence")
    ],
    title="📰 Fake News Detector + Explainer",
    description="Classifies a news article as FAKE or REAL and explains using Gemma + Wikipedia."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc9a2e7e6a1a9b3474.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
print(query_gemma("Say hello world"))


Hello world!

